In [1]:
import pickle
import gc
from datetime import datetime
from os.path import exists, join

import numpy as np
import pandas as pd
import scipy as sp

# import matplotlib.pyplot as plt

from sklearn import model_selection
from sklearn.model_selection import KFold
from sklearn.model_selection import train_test_split

import xgboost as xgb
from xgboost import XGBClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn import model_selection
from sklearn.metrics import mean_squared_error, mean_absolute_error

/home/zhanglun/.pyenv/versions/anaconda3-4.1.0/lib/python3.5/site-packages/sklearn/cross_validation.py:44: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)


In [2]:
train_f = pd.read_pickle('../dataset/train/train_all_feature.pkl')
train_f.shape

(879539, 443)

In [3]:
test_f = pd.read_pickle('../dataset/train/test_all_feature.pkl')
test_f.shape

(215287, 443)

In [4]:
luck = 666

In [5]:
parameters = {
#     'gamma': [0.05],
    #     'n_estimators': [800],
    'max_depth': [2],
#     'learning_rate': [0.15, 0.3],
#         'subsample': [0.9],
#         'colsample_bytree': [0.9],
#     'min_child_weight': [1, 2],
    #     'reg_alpha': [0, 1, 5],
    #     'reg_lambda': [0, 1, 4],
}

In [6]:
model_path = '../dataset/xgbmodel.pkl'

In [7]:
model = XGBClassifier(nthread=4, silent=False, objective= 'binary:logistic', n_estimators=100, seed=luck)

In [8]:
clf = model_selection.GridSearchCV(model, parameters, n_jobs=-1, cv=2, verbose=True)

In [9]:
use_columns = [
    x for x in train_f.columns
    if x not in [
        'orderid', 'uid', 'hotelid', 'basicroomid', 'hotel_roomid', 'roomid',
        'orderlabel'
    ]
]

In [10]:
train_f.shape

(879539, 443)

In [11]:
clf_feture = train_f.sample(5000, random_state=luck)

In [12]:
%time clf.fit(clf_feture[use_columns], clf_feture['orderlabel'])

Fitting 2 folds for each of 1 candidates, totalling 2 fits


[Parallel(n_jobs=-1)]: Done   2 out of   2 | elapsed:    4.5s finished


CPU times: user 12.9 s, sys: 97.8 ms, total: 13 s
Wall time: 8.15 s


GridSearchCV(cv=2, error_score='raise',
       estimator=XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=1,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=3,
       min_child_weight=1, missing=None, n_estimators=100, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=666, silent=False, subsample=1),
       fit_params={}, iid=True, n_jobs=-1, param_grid={'max_depth': [2]},
       pre_dispatch='2*n_jobs', refit=True, return_train_score=True,
       scoring=None, verbose=True)

In [13]:
clf.best_score_

0.96999999999999997

In [14]:
# # # clf.best_params_['n_estimators'] = 
clf.best_params_['max_depth'] = 6
# # clf.best_params_['gamma'] = 2
clf.best_params_['subsample'] = 0.8
clf.best_params_['colsample_bytree'] = 0.8
# # clf.best_params_['n_estimators'] = 1000
clf.best_params_['learning_rate'] = 0.1
# # clf.best_params_['min_child_weight'] = 2
# # clf.best_params_['reg_alpha'] = 0.1

# clf.best_params_['scale_pos_weight'] = 5





In [15]:
model = XGBClassifier(
    nthread=4,
    silent=False,
    objective='binary:logistic',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=1000,
    **clf.best_params_)
model
# model 

XGBClassifier(base_score=0.5, colsample_bylevel=1, colsample_bytree=0.8,
       gamma=0, learning_rate=0.1, max_delta_step=0, max_depth=6,
       min_child_weight=1, missing=None, n_estimators=1000, nthread=4,
       objective='binary:logistic', reg_alpha=0, reg_lambda=1,
       scale_pos_weight=1, seed=666, silent=False, subsample=0.8)

In [16]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel']),
             (test_f[use_columns],
    test_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
    early_stopping_rounds=30,
)

[0]	validation_0-auc:0.899755	validation_1-auc:0.897268
Multiple eval metrics have been passed: 'validation_1-auc' will be used for early stopping.

Will train until validation_1-auc hasn't improved in 30 rounds.
[1]	validation_0-auc:0.907763	validation_1-auc:0.906011
[2]	validation_0-auc:0.909966	validation_1-auc:0.90784
[3]	validation_0-auc:0.911352	validation_1-auc:0.909017
[4]	validation_0-auc:0.912318	validation_1-auc:0.909981
[5]	validation_0-auc:0.913015	validation_1-auc:0.910467
[6]	validation_0-auc:0.9135	validation_1-auc:0.910766
[7]	validation_0-auc:0.91472	validation_1-auc:0.911962
[8]	validation_0-auc:0.914966	validation_1-auc:0.91219
[9]	validation_0-auc:0.915321	validation_1-auc:0.912559
[10]	validation_0-auc:0.915493	validation_1-auc:0.912886
[11]	validation_0-auc:0.915733	validation_1-auc:0.913028
[12]	validation_0-auc:0.916288	validation_1-auc:0.913356
[13]	validation_0-auc:0.916898	validation_1-auc:0.913837
[14]	validation_0-auc:0.917126	validation_1-auc:0.914173
[15

KeyboardInterrupt: 

In [ ]:
def get_score(model, train_f):
    result = model.predict_proba(train_f[use_columns])[:, 1]
    train_f['prob'] = result
    train_x = train_f[['orderid', 'prob', 'orderlabel']]
    train_x.sort_values('prob', ascending=False, inplace=True)
    finall_result_x = train_x.drop_duplicates(['orderid'])
    train_score = finall_result_x.orderlabel.mean()
    print(train_score)
    return train_score

In [ ]:
train_score = get_score(model, train_f)

In [ ]:
test_score = get_score(model, test_f)

In [ ]:
acu = model.score(test_f[use_columns], test_f['orderlabel'])
acu

In [ ]:
model.best_ntree_limit

In [ ]:
if not exists('models'):
    mkdir('models')
model_importance_path = join('models', datetime.now().strftime('%d-%H%M-xgbimportance.txt'))
model_importance_path

In [ ]:
importance_df = pd.DataFrame(model.feature_importances_, index=use_columns)

importance_df.sort_values(0, ascending=False, inplace=True)
importance_df.to_csv(model_importance_path)

In [ ]:
model.get_params()

In [ ]:
parms = ','.join(['{}: {}'.format(k, v) for k, v in model.get_params().items()])
parms

In [ ]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.evals_result()['validation_0']['auc'][model.best_iteration],
    'model_test',
    model.evals_result()['validation_1']['auc'][model.best_iteration],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep='  ')

In [ ]:
print(
    datetime.now().strftime('%d-%H-%M:'),
    'online',
    '??',
    'test_score',
    test_score,
    'train_score',
    train_score,
    'model_train',
    model.evals_result()['validation_0']['auc'][model.best_iteration],
    'model_test',
    model.evals_result()['validation_1']['auc'][model.best_iteration],
    'accu',
    acu,
    'n',
    model.best_iteration,
    'params',
    parms,
    'shape', train_f.shape,
    end='\n',
    sep=',', file=open('xgb_result.txt', 'a+'))

In [ ]:
pickle.dump(model, open(model_path, 'wb'))

In [ ]:
del train_f, test_f

gc.collect()

In [ ]:
train_f = pd.read_pickle('../dataset/train/all_feature.pkl')
train_f.shape

In [ ]:
n_est = model.best_iteration + 300 if model.best_iteration > 0 else 1500

In [ ]:
model = XGBClassifier(
    nthread=4,
    silent=False,
    objective='binary:logistic',
    seed=luck,
#     learning_rate=0.3,
    n_estimators=n_est,
    **clf.best_params_)
model
# model 

In [ ]:
model.fit(
    train_f[use_columns],
    train_f['orderlabel'],
    eval_set=[(train_f[use_columns],
    train_f['orderlabel'])],
    eval_metric='auc',
    verbose=True,
)

In [ ]:
pickle.dump(model, open(model_path, 'wb'))